In [1]:
import datasets

datasets.downloads = '/home/greg/Downloads/'

trX, teX, trY, teY = datasets.mnist()

In [2]:
import numpy as np
from softmax import softmax

In [3]:
X = trX
Y = trY

epochs = 1
learning_rate = 0.05
H = 625

N, D = X.shape
N, C = Y.shape

In [6]:
import theano
from theano import tensor as T

x = T.fmatrix()
y = T.fmatrix()

floatX = lambda X: np.asarray(X, dtype = theano.config.floatX)
weights = lambda n, d: theano.shared(floatX(np.random.randn(n, d) * 0.01))

w_h = weights(D, H)
w_o = weights(H, C)

h = T.nnet.sigmoid(T.dot(x, w_h))
mu = T.nnet.softmax(T.dot(h, w_o))

cost = T.mean(T.nnet.categorical_crossentropy(mu, y))
params = [w_h, w_o]
grads = T.grad(cost, params)
updates = [[p, p - g * learning_rate] for p, g in zip(params, grads)]

train = theano.function([x, y], updates = updates, allow_input_downcast = True)
predict = theano.function([x], T.argmax(mu, axis = 1), allow_input_downcast = True)

for _ in range(epochs):
    for start, end in zip(range(0, N, 128), range(128, N, 128)):
        train(X[start : end], Y[start : end])
        
w_h, w_o = w_h.get_value(), w_o.get_value()
yhat = np.argmax(X.dot(w_h).dot(w_o), axis = 1)        
print np.mean(np.argmax(Y, axis = 1) == predict(X))
print np.mean(yhat == predict(X))

0.698783333333
0.642333333333


In [10]:
import tensorflow as tf

x = tf.placeholder('float')
y = tf.placeholder('float')

weights = lambda shape: tf.Variable(tf.random_normal(shape))

w = weights([D, H])
b = weights([H])
l = tf.nn.relu(tf.add(tf.matmul(x, w), b))

w = weights([H, C])
b = weights([C])
model = tf.matmul(l, w) + b

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(model, y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())

    for _ in range(epochs):
        for start, end in zip(range(0, N, 100), range(100, N, 100)):
            feed_dict = {x: X[start : end], y: Y[start : end]}
            sess.run(optimizer, feed_dict = feed_dict)

    yhat = tf.argmax(model, 1).eval({x: X, y: Y})
    print np.mean(yhat == np.argmax(Y, axis = 1))

0.921716666667
